# Final Project: Fake News Detection

By Felix Daubner - Hochschule der Medien

Module 'Supervised and Unsupervised Learning' - Prof. Dr.-Ing. Johannes Maucher

## Model Training

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Flatten, Dense, LSTM, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional, Input, Concatenate
import pickle
from gensim.models import KeyedVectors

NUM_WORDS=3000
MAX_SEQUENCE_LEN = 57
NUM_CAT = 20

In [163]:
def prepareFeatures(X):
    X_token = np.array(X["token"].apply(np.asarray))
    X_token = np.array([arr for arr in X_token])

    X_enc = np.array(X.drop(["token"], axis=1).apply(np.array))

    return X_token, X_enc

def prepareTarget(y):
    return np.array(y)    

This section contains the model training and feature selection. Different types of models should be trained and then compared to find out which model fits the challenge, to determine whether a political statement was fake-news or true, best. There are three types of models to be compared: MLP, CNN and LSTM. Those models should also vary in terms of hyperparameters like layers, neurons, optimization and else. The best model is evaluated and then optimized in the next section.

### Prepare data for training and validation

In [2]:
data = pd.read_json("data/processed.json", orient="records", lines=True)

In [3]:
data.columns

Index(['statement', 'channel_Instagram', 'channel_Other', 'channel_TV',
       'channel_TikTok', 'channel_X', 'channel_ad', 'channel_article',
       'channel_blog', 'channel_campaign', 'channel_debate',
       'channel_interview', 'channel_lecture', 'channel_mail',
       'channel_podcast', 'channel_presentation', 'channel_press',
       'channel_social media', 'channel_speech', 'channel_talk',
       'channel_video', 'truth', 'token', 'statement_stop', 'token_stop'],
      dtype='object')

Before starting defining the different models, the data is prepared for the training process. The neural network to be trained only takes numpy arrays as input. Thus, the data currently saved as a pandas DataFrame is converted in to a numpy array. In this conversion process, only "token", the encoded channel and issue columns and "truth" are kept meaning column 'statement' is dropped as it can not be used for model training.

After splitting the data into features and target, the features still have to preprared for training by splitting the encoded categorical data from the tokenized and padded statements. The statement data has to be taken care of using an Embedding Layer while a Dense layer is sufficient to handle the encoded categorical data.

In [4]:
X = data.drop(["statement", "statement_stop", "token_stop", "truth"], axis=1)
y = data["truth"]

In [82]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=42)

In [164]:
X_train_token, X_train_enc = prepareFeatures(X_train)
X_val_token, X_val_enc = prepareFeatures(X_val)
y_train = prepareTarget(y_train)
y_val = prepareTarget(y_val)

### Prepare data for testing

In [7]:
test = pd.read_json("data/LIAR_processed.json", orient="records", lines=True)

In [167]:
X_test = test.drop(["statement", "statement_stop", "token_stop", "truth"], axis=1)
y_test = test["truth"]

In [168]:
X_test_token, X_test_enc = prepareFeatures(X_test)
y_test = prepareTarget(y_test)

When looking at fake-news detection, it is decided which of accuracy, precision or recall should be optimized. Most often, accuracy is not a good metric as it doesn't include the cost of mis-predicting. That's why either precision or recall should be used.
The worst case at fake-news is when a fake-news is not identified as fake-news. Whereas the other way, a true statement being classified as fake-news does not harm in the same way. Translating this into the terms of this project means a false positive ("a statement which is 'fake' (0) gets classified as 'true' (1)") is worse than a false negative ("a statement which is 'true' (1) gets classified as 'false' (0)"). The metrics focusing on optimizing the false positives is precision.

In the following, four different types models are trained and evaluated. Based on those evaluations, the best model is chosen and optimized until pre-defined metrics reach their peak. The evaluation for the best model are done using all available features of the data. In the following section [Optimiziation](07_evaluation-optimization.ipynb) the most useful features and hyperparameters of the model are chosen until the best model is reached.

### Prepare infrastructure

In [9]:
word2vec = KeyedVectors.load_word2vec_format("wiki-news-300d-1M.vec")

In [10]:
with open("tokenizer/tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)

In [11]:
embedding_dim = 300  
word_index = tokenizer.word_index 
num_words = min(len(word_index) + 1, NUM_WORDS)  

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < num_words:
        if word in word2vec.key_to_index:
            embedding_vector = word2vec[word]
            embedding_matrix[i] = embedding_vector

In [169]:
text_input = Input(shape=(MAX_SEQUENCE_LEN,), name="text_input")
categorical_input = Input(shape=(NUM_CAT,), name="categorical_input")

In [170]:
emb = Embedding(NUM_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LEN, trainable=False)(text_input)

In [171]:
cat = Dense(32, activation="relu")(categorical_input)

### Feedforward Neural Nerwork

In [186]:
ff_flatten_text = Flatten()(emb)

ff_combined = Concatenate()([ff_flatten_text, cat])
ff_dense1 = Dense(128, activation="relu")(ff_combined)
ff_dense2 = Dense(64, activation="relu")(ff_dense1)
ff_output = Dense(1, activation="sigmoid")(ff_dense2)

In [187]:
ff = Model(inputs=[categorical_input, text_input], outputs=ff_output)
ff.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 57)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 57, 300)      900000      ['text_input[0][0]']             
                                                                                                  
 categorical_input (InputLayer)  [(None, 20)]        0           []                               
                                                                                                  
 flatten_9 (Flatten)            (None, 17100)        0           ['embedding_4[0][0]']            
                                                                                            

In [188]:
ff.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.Precision()])

In [189]:
ff_history = ff.fit([X_train_enc, X_train_token], y_train, epochs=20, batch_size=128, validation_data=([X_val_enc, X_val_token], y_val))

Epoch 1/20
111/111 [==============================] - 2s 11ms/step - loss: 0.5448 - precision_5: 0.7067 - val_loss: 0.4498 - val_precision_5: 0.7915
Epoch 2/20
111/111 [==============================] - 1s 11ms/step - loss: 0.3195 - precision_5: 0.8558 - val_loss: 0.3925 - val_precision_5: 0.8134
Epoch 3/20
111/111 [==============================] - 1s 11ms/step - loss: 0.1451 - precision_5: 0.9457 - val_loss: 0.3793 - val_precision_5: 0.8655
Epoch 4/20
111/111 [==============================] - 1s 11ms/step - loss: 0.0523 - precision_5: 0.9869 - val_loss: 0.4322 - val_precision_5: 0.8519
Epoch 5/20
111/111 [==============================] - 1s 11ms/step - loss: 0.0139 - precision_5: 0.9985 - val_loss: 0.5355 - val_precision_5: 0.8672
Epoch 6/20
111/111 [==============================] - 1s 11ms/step - loss: 0.0033 - precision_5: 0.9999 - val_loss: 0.6107 - val_precision_5: 0.8511
Epoch 7/20
111/111 [==============================] - 1s 11ms/step - loss: 0.0016 - precision_5: 0.9999 - 

### LSTM

In [190]:
lstm_ = LSTM(64)(emb)

In [191]:
lstm_combined = Concatenate()([lstm_, cat])

In [192]:
lstm_dense1 = Dense(64, activation='relu')(lstm_combined)
lstm_drop1 = Dropout(0.2)(lstm_dense1)
lstm_dense2 = Dense(32, activation='relu')(lstm_drop1)
lstm_drop2 = Dropout(0.4)(lstm_dense2)
lstm_output = Dense(1, activation='sigmoid')(lstm_drop2)

In [193]:
lstm = Model(inputs=[categorical_input, text_input], outputs=lstm_output)
lstm.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 57)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 57, 300)      900000      ['text_input[0][0]']             
                                                                                                  
 categorical_input (InputLayer)  [(None, 20)]        0           []                               
                                                                                                  
 lstm_7 (LSTM)                  (None, 64)           93440       ['embedding_4[0][0]']            
                                                                                            

In [194]:
lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.Precision()])

In [195]:
lstm_hist = lstm.fit([X_train_enc, X_train_token], y_train, batch_size=128, epochs=20, validation_data=([X_val_enc, X_val_token], y_val))

Epoch 1/20
111/111 [==============================] - 8s 57ms/step - loss: 0.6170 - precision_6: 0.6387 - val_loss: 0.5513 - val_precision_6: 0.6711
Epoch 2/20
111/111 [==============================] - 6s 53ms/step - loss: 0.5518 - precision_6: 0.6866 - val_loss: 0.5379 - val_precision_6: 0.6642
Epoch 3/20
111/111 [==============================] - 6s 56ms/step - loss: 0.5312 - precision_6: 0.7037 - val_loss: 0.5256 - val_precision_6: 0.6939
Epoch 4/20
111/111 [==============================] - 6s 55ms/step - loss: 0.5156 - precision_6: 0.7181 - val_loss: 0.5209 - val_precision_6: 0.6883
Epoch 5/20
111/111 [==============================] - 6s 53ms/step - loss: 0.4935 - precision_6: 0.7262 - val_loss: 0.5188 - val_precision_6: 0.6686
Epoch 6/20
111/111 [==============================] - 6s 54ms/step - loss: 0.4711 - precision_6: 0.7387 - val_loss: 0.4976 - val_precision_6: 0.7174
Epoch 7/20
111/111 [==============================] - 6s 55ms/step - loss: 0.4475 - precision_6: 0.7550 - 

### Bi-directional LSTM

In [196]:
blstm_ = Bidirectional(LSTM(128))(emb)

In [197]:
blstm_combined = Concatenate()([blstm_, cat])

In [198]:
blstm_dense1 = Dense(64, activation='relu')(blstm_combined)
blstm_drop1 = Dropout(0.2)(blstm_dense1)
blstm_output = Dense(1, activation='sigmoid')(blstm_drop1)


In [199]:
blstm = Model(inputs=[categorical_input, text_input], outputs=blstm_output)
blstm.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 57)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 57, 300)      900000      ['text_input[0][0]']             
                                                                                                  
 categorical_input (InputLayer)  [(None, 20)]        0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 256)         439296      ['embedding_4[0][0]']            
 )                                                                                          

In [200]:
blstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.Precision()])

In [201]:
blstm_hist = blstm.fit([X_train_enc, X_train_token], y_train, batch_size=128, epochs=20, validation_data=([X_val_enc, X_val_token], y_val))

Epoch 1/20
111/111 [==============================] - 15s 118ms/step - loss: 0.5906 - precision_7: 0.6647 - val_loss: 0.5501 - val_precision_7: 0.6678
Epoch 2/20
111/111 [==============================] - 14s 130ms/step - loss: 0.5361 - precision_7: 0.7020 - val_loss: 0.5410 - val_precision_7: 0.7326
Epoch 3/20
111/111 [==============================] - 13s 118ms/step - loss: 0.5197 - precision_7: 0.7123 - val_loss: 0.5254 - val_precision_7: 0.7003
Epoch 4/20
111/111 [==============================] - 13s 120ms/step - loss: 0.4932 - precision_7: 0.7311 - val_loss: 0.5159 - val_precision_7: 0.6907
Epoch 5/20
111/111 [==============================] - 13s 119ms/step - loss: 0.4655 - precision_7: 0.7479 - val_loss: 0.5006 - val_precision_7: 0.7011
Epoch 6/20
111/111 [==============================] - 13s 122ms/step - loss: 0.4282 - precision_7: 0.7650 - val_loss: 0.5428 - val_precision_7: 0.7645
Epoch 7/20
111/111 [==============================] - 13s 120ms/step - loss: 0.3996 - precisio

### Convolutional Neural Network

In [208]:
cnn_ = Conv1D(filters=128, kernel_size=5, activation='relu')(emb)
cnn_maxpool = MaxPooling1D(pool_size=5)(cnn_)

In [209]:
cnn_flatten_text = Flatten()(cnn_)

cnn_combined = Concatenate()([cnn_flatten_text, cat])
cnn_flatten = Flatten()(cnn_maxpool)
cnn_dense1 = Dense(128, activation="relu")(cnn_flatten)
cnn_output = Dense(1, activation="sigmoid")(cnn_dense1)

In [210]:
cnn = Model(inputs=[categorical_input, text_input], outputs=cnn_output)
cnn.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 57)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 57, 300)      900000      ['text_input[0][0]']             
                                                                                                  
 conv1d_3 (Conv1D)              (None, 53, 128)      192128      ['embedding_4[0][0]']            
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 10, 128)     0           ['conv1d_3[0][0]']               
                                                                                           

In [211]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.Precision()])

In [213]:
cnn_hist = cnn.fit([X_train_enc, X_train_token], y_train, batch_size=128, epochs=20, validation_data=([X_val_enc, X_val_token], y_val))

Epoch 1/20
111/111 [==============================] - 3s 23ms/step - loss: 0.5922 - precision_8: 0.6826 - val_loss: 0.5114 - val_precision_8: 0.7719
Epoch 2/20
111/111 [==============================] - 3s 23ms/step - loss: 0.3849 - precision_8: 0.8329 - val_loss: 0.3838 - val_precision_8: 0.8140
Epoch 3/20
111/111 [==============================] - 3s 24ms/step - loss: 0.1814 - precision_8: 0.9367 - val_loss: 0.4162 - val_precision_8: 0.8091
Epoch 4/20
111/111 [==============================] - 3s 25ms/step - loss: 0.0749 - precision_8: 0.9808 - val_loss: 0.4114 - val_precision_8: 0.8578
Epoch 5/20
111/111 [==============================] - 2s 22ms/step - loss: 0.0336 - precision_8: 0.9941 - val_loss: 0.4418 - val_precision_8: 0.8717
Epoch 6/20
111/111 [==============================] - 2s 22ms/step - loss: 0.0210 - precision_8: 0.9968 - val_loss: 0.4727 - val_precision_8: 0.8696
Epoch 7/20
111/111 [==============================] - 2s 22ms/step - loss: 0.0154 - precision_8: 0.9980 - 